In [2]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))

"/home/twan/.julia/dev/PushRecovery/Project.toml"

In [2]:
using PushRecovery
using PushRecovery.ConvexHull
using StaticArrays

┌ Info: Precompiling PushRecovery [e5d7e63e-020d-11e9-1e2d-c1b17e981caa]
└ @ Base loading.jl:1189


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/29d4a42ceb6d03d53ba5feb8981c0eb5bceb5139-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [3]:
using MathOptInterface
const MOI = MathOptInterface
using OSQP
using OSQP.MathOptInterfaceOSQP

atol_distance = 1e-3
optimizer = OSQP.Optimizer()
MOI.set(optimizer, OSQPSettings.Verbose(), false)
MOI.set(optimizer, OSQPSettings.EpsAbs(), atol_distance^2)
problem = ConvexHullProblem{2, 10, Float64}(optimizer);

In [4]:
using BenchmarkTools

In [6]:
for i = 1 : 10
    set_point!(problem, rand(SVector{2, Float64}))
    set_vertices!(problem, [rand(SVector{2, Float64}) for i = 1 : 10])
    solve!(problem)
    @show is_point_inside(problem; atol=atol_distance)
    @show distance_to_closest_point(problem)
    @show closest_point(problem)
    println()
end

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.03486977218524078
closest_point(problem) = [0.0217751, 0.00336563]

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.11519197245072557
closest_point(problem) = [0.11119, 0.370043]

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.6307925730210995
closest_point(problem) = [0.242595, 0.367315]

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.6752080941431933
closest_point(problem) = [0.655103, 0.254993]

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.05684085081661188
closest_point(problem) = [0.810345, 0.662103]

is_point_inside(problem; atol=atol_distance) = false
distance_to_closest_point(problem) = 0.3047256579999386
closest_point(problem) = [0.689921, 0.706642]

is_point_inside(problem; atol=atol_distance) = false
distan

In [7]:
@btime solve!($problem) setup = begin
    set_point!($problem, rand(SVector{2, Float64}))
    set_vertices!($problem, [rand(SVector{2, Float64}) for i = 1 : 10])
end

  22.644 μs (0 allocations: 0 bytes)


In [10]:
vertices = [rand(SVector{2, Float64}) for i = 1 : 10]
@btime set_point!($problem, zero(SVector{2}))

  1.768 ns (0 allocations: 0 bytes)


2-element MArray{Tuple{2},Float64,1,2}:
 0.0
 0.0

In [17]:
SizedVector{10}(vertices)

10-element SizedArray{Tuple{10},SArray{Tuple{2},Float64,1,2},1,1}:
 [0.609003, 0.302246]
 [0.224927, 0.803468]
 [0.922636, 0.808817]
 [0.351722, 0.314121]
 [0.425197, 0.133761]
 [0.318272, 0.596267]
 [0.125196, 0.790781]
 [0.637986, 0.370337]
 [0.768763, 0.989335]
 [0.918016, 0.762583]

MathOptInterface

In [37]:
module ConvexHullTest

using PushRecovery.ConvexHull
using StaticArrays
using MathOptInterface
using OSQP
using OSQP.MathOptInterfaceOSQP
using Test
using Random
using LinearAlgebra
using BenchmarkTools

const MOI = MathOptInterface

function optimizer(atol_distance)
    optimizer = OSQP.Optimizer()
    MOI.set(optimizer, OSQPSettings.Verbose(), false)
    MOI.set(optimizer, OSQPSettings.EpsAbs(), atol_distance^2)
    MOI.set(optimizer, OSQPSettings.EpsRel(), 1e-8)
#     MOI.set(optimizer, OSQPSettings.Polish(), true)
#     MOI.set(optimizer, OSQPSettings.AdaptiveRhoInterval(), 25) # required for deterministic behavior
    optimizer
end

@testset "ConvexHull basics" begin
    atol_distance = 1e-3
    problem = ConvexHullProblem{2, 3, Float64}(optimizer(atol_distance))
    set_vertices!(problem, [SVector(0., 0.), SVector(2., 0.), SVector(0., 3.)])

    p = SVector(0.5, 0.5)
    set_point!(problem, p)
    solve!(problem)
    @test is_point_inside(problem; atol=atol_distance)
    @test distance_to_closest_point(problem) ≈ 0 atol=atol_distance
    @test closest_point(problem) ≈ p atol=atol_distance

    p = SVector(-0.5, -0.5)
    set_point!(problem, p)
    solve!(problem)
    @test !is_point_inside(problem; atol=atol_distance)
    @test distance_to_closest_point(problem) ≈ sqrt(0.5) atol=atol_distance
    @test closest_point(problem) ≈ SVector(0.0, 0.0) atol=atol_distance
end

@testset "ConvexHull point inside" begin
    atol_distance = 1e-3
    N = 2
    M = 10
    problem = ConvexHullProblem{N, M, Float64}(optimizer(atol_distance))
    Random.seed!(1)
    for i = 1 : 10000
        vertices = [rand(SVector{N}) for i = 1 : M]
        set_vertices!(problem, vertices)
        weights = rand(M)
        weights ./= sum(weights)
        point = sum(weights .* vertices)
        set_point!(problem, point)
        solve!(problem)
        @test is_point_inside(problem; atol=atol_distance)
    end
end

@testset "ConvexHull point outside" begin
    atol_distance = 1e-3
    N = 2
    M = 10
    problem = ConvexHullProblem{N, M, Float64}(optimizer(atol_distance))
    Random.seed!(1)
    for i = 1 : 10000
        set_vertices!(problem, [(1 - atol_distance) * normalize(randn(SVector{N})) for i = 1 : M])
        point = normalize(randn(SVector{N}))
        set_point!(problem, point)
        solve!(problem)
        @test !is_point_inside(problem; atol=atol_distance)
        closest = closest_point(problem)
        @test norm(point - closest) ≈ distance_to_closest_point(problem) atol=atol_distance
        
        set_point!(problem, closest)
        solve!(problem)
        @test is_point_inside(problem; atol=atol_distance)
        @test distance_to_closest_point(problem) ≈ 0 atol=atol_distance
    end
end

@testset "ConvexHull allocations" begin
    atol_distance = 1e-3
    N = 2
    M = 10
    problem = ConvexHullProblem{N, M, Float64}(optimizer(atol_distance))
    vertices = [rand(SVector{N}) for i = 1 : M]
    point = rand(SVector{N})
    
    testfun = function (problem, vertices, point)
        set_vertices!(problem, vertices)
        set_point!(problem, point)
        solve!(problem)
    end
    
    testfun(problem, vertices, point)
    allocs = @allocated testfun(problem, vertices, point)
    @test allocs == 0
    @btime $testfun($problem, $vertices, $point)
    
end

end

Test Summary:     | Pass  Total
ConvexHull basics |    6      6
Test Summary:           | 

 Pass  Total
ConvexHull point inside | 10000  10000
Test Summary:            |  Pass  Total
ConvexHull point outside | 40000  40000
  24.454 μs (0 allocations: 0 bytes)
Test Summary:          | Pass  Total
ConvexHull allocations |    1      1


Main.ConvexHullTest